# Automated Landsat 8 image bulk download and preprocessing

_Last modified 2022-01-07._

This script is run to download and preprocess the Landsat 8 images over the glaciers prior to the Wavelet Transform Modulus Maxima (WTMM) segmentation analysis where the calving front delineations are produced.

The code is streamlined to analyze images for hundreds of glaciers, specifically, the marine-terminating glaciers along the periphery of Greenland. For use on other glaciers, sections of code must be modified:

##########################################################################################

__Indicates code that must be modified__

##########################################################################################

Recommendation is to keep a record of the csv file names generated throughout the preprocessing as many of them will be used later for analysis.

Terminal command line requirements:

 - Contains code to bulk download Landsat 8 images and metadata stored on the Amazon Web Services cloud (s3 bucket) over a region of interest using the Amazon Web Services command line interface. Follow instructions at https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-install.html to get required __aws__ commands onto your command terminal.
 
 - Uses GDAL command line functions. Make sure __gdal__ is installed properly.
 - Uses image magick command line functions. Download instructions available at https://imagemagick.org/script/download.php.


Outline of steps:
1. Set-up: import packages, set paths, and enter glaciers IDs
2. Find all the Landsat footprints that overlap the glaciers
3. Download Landsat metadata (*MTL.txt) files from AWS for all overlapping scenes
4. Calculate cloud % over terminus box using Landsat quality band
5. Create buffer zone around terminus boxes and rasterize terminus boxes
6. Download non-cloudy Landsat images from AWS
7. Calculate weighted average glacier flow direction using velocity data
8. Rotate all images by flow direction
9. Crop all images to the same size

# 1) Set-up: import packages, set paths, and enter glaciers IDs

In [1]:
! aws s3 ls --profile terminusmapping

In [86]:
import numpy as np
import pandas as pd
import scipy
import math
import subprocess
import os
import shutil
import datetime
import cv2
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob


# geospatial packages
import fiona
import geopandas as gpd
from shapely.geometry import Polygon, Point, LineString
import shapely
from matplotlib.pyplot import imshow
import rasterio as rio
from rasterio.session import AWSSession

# AWS packages
import pickle
import boto3
import boto3.session

cred = boto3.Session().get_credentials()
ACCESS_KEY = cred.access_key
SECRET_KEY = cred.secret_key
SESSION_TOKEN = cred.token  ## optional

s3client = boto3.client('s3', 
                        aws_access_key_id = ACCESS_KEY, 
                        aws_secret_access_key = SECRET_KEY, 
                        aws_session_token = SESSION_TOKEN
                       )

# response = s3client.get_object(Bucket='name_of_your_bucket', Key='path/to_your/file.pkl')

# body = response['Body'].read()
# data = pickle.loads(body)


# Enable fiona KML file reading driver
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'


######################################################################################
# ADJUST THESE VARIABLES:
basepath = '/Users/phoebekinzelman/research/greenland/updated_shps/' # folder containing the terminus box shapefile(s)
downloadpath ='/Users/phoebekinzelman/research/greenland/LS8aws/' # folder to eventually contain downloaded Landsat 5 / 7 images
os.chdir('/Users/phoebekinzelman/research/greenland/repo/automated-glacier-terminus/') # path to this repo
years = ['2013','2014','2015','2016','2017','2018','2019','2020','2021']
######################################################################################

# import necessary functions from automated-glacier-terminus.py
from automated_terminus_functions import distance, resize_pngs

In [3]:
# change display width
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [4]:
# !pip install scipy

### Enter in the glacier BoxIDs:

The Greenland peripheral glacier terminus boxes were referenced using their 3 digit BoxID: Box###.
For other glaciers, replace this code with a list of IDs corresponding to the glaciers and corresponding shapefiles (e.g. BoxHelheim.shp). 

In [5]:
######################################################################################

BoxIDs = []
boxes = list(map(str, np.arange(8, 9, 1))) #1, 642, 1
for BoxID in boxes:
    BoxID = BoxID.zfill(3)
    BoxIDs.append(BoxID)
print(BoxIDs)

# BoxIDs = ['Kangerlussuaq','Bernstorff1','Bernstorff2','Bernstorff3']
# BoxIDs = ['HM','Petermann']
# print(BoxIDs)
######################################################################################

['008']


### Create new folders corresponding to these glaciers:

In [6]:
# BOX 008 DELETES HERE
# create new BoxID folders 
for BoxID in BoxIDs:
    # create folder in updated_shps
    if os.path.exists(basepath+'Box'+BoxID):
#         shutil.rmtree(basepath+'Box'+BoxID) # remove the old folder
        print("Path exists already for Box", BoxID)
    else:
        os.mkdir(basepath+'Box'+BoxID)
            
    # make a new folder for the glacier in the download folder
    if os.path.exists(downloadpath+'Box'+BoxID):
        print("Path exists already in LS8aws for Box", BoxID)
    else:
        os.mkdir(downloadpath+'Box'+BoxID)

#     ######################################################################################
#     # ONLY APPLIES TO GREENLAND PERIPHERAL GLACIERS, MUST MODIFY FOR OTHER GLACIERS
#     # OR COMMENT OUT AND DO MANUALLY
    boxespath = basepath # path to terminus box shapefiles (updated_shps)
    RGIpath = '/Users/phoebekinzelman/research/greenland/RGI_shps/' # path to RGI glacier outlines
    
    # move terminus box shapefile into the new folder
    ID = int(BoxID) # make into an integer in order to grab the .shp files from updated_shps  
    for file in os.listdir(boxespath):
        if file.startswith(str(ID)):
            if len(file) == len(str(ID))+4:
#              if file.endswith('.dbf') or file.endswith('.prj') or file.endswith('.qpj') or file.endswith('.shx') or file.endswith('.shp'):
                shutil.copyfile(boxespath+file, basepath+'Box'+BoxID+'/Box'+BoxID+file[-4:])

    # move RGI glacier outline into the new folder
    for file in os.listdir(RGIpath):
        if file.startswith('BoxID_'+str(ID)):
            shutil.copyfile(RGIpath+file, basepath+'Box'+BoxID+'/RGI_Box'+BoxID+file[-4:])
            
    ######################################################################################

Path exists already for Box 008
Path exists already in LS8aws for Box 008


# 2) Find all the Landsat footprints that overlap the glaciers

This step requires the WRS-2_bound_world_0.kml file containing the footprints of all the Landsat scene boundaries available through the USGS (https://www.usgs.gov/land-resources/nli/landsat/landsat-shapefiles-and-kml-files). Place this file in your base directory (basepath). 

To check if they overlap the glacier terminus box shapefiles, the box shapefiles must be in WGS84 coordinates (ESPG: 4326). 
 - If they are already in WGS84, skip this step and rename them to end in "_WGS.shp"
 - If they are not, we can use the following GDAL command to reproject:

        ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:NEW_EPSG_NUMBER -s_srs EPSG:OLD_EPSG_NUMBER output.shp input.shp

In [7]:
# Reproject terminus box shapefiles to WGS84 if in a different projection
######################################################################################
source_srs = 'EPSG:3413' # current projection of the terminus box shapefiles (Greenland polar stereo = 3413)
######################################################################################

for BoxID in BoxIDs:
    boxespath = basepath+"Box"+BoxID+"/Box"+BoxID # access the BoxID folders created 
    # construct the gdal command
    rp = "ogr2ogr -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs "+source_srs+" "+boxespath+"_WGS.shp "+boxespath+".shp"
    print(rp)
    subprocess.call(rp, shell=True) # run the command on terminal
    print("Box"+BoxID+" done.") # keep track of progress

ogr2ogr -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs EPSG:3413 /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_WGS.shp /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008.shp
Box008 done.


In [8]:
# Grab the WGS84 coordinates of the boxes
box_points = {} # dictionary of points
for BoxID in BoxIDs:
    boxpath = basepath+"Box"+BoxID+"/Box"+BoxID
    termbox = fiona.open(boxpath+'_WGS.shp') # open reprojected terminus boxes
    box = termbox.next(); box_geom= box.get('geometry'); box_coords = box_geom.get('coordinates')[0] # grab coords
    points = [] # to hold the box vertices
    
    for coord_pair in box_coords:
        lat = coord_pair[0]; lon = coord_pair[1]        
        point = shapely.geometry.Point(lat, lon) # create shapely point 
        points.append(point) # append to points list
        
    box_points.update({BoxID: points}) # update dictionary
    print("Box"+BoxID+" done.") # keep track of progress

Box008 done.


/var/folders/kw/nmc8p1t51rq1bkcn6xrr0yv40000gr/T/ipykernel_61071/2258550550.py:6: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  box = termbox.next(); box_geom= box.get('geometry'); box_coords = box_geom.get('coordinates')[0] # grab coords


In [9]:
# open the kml file with the Landsat path, row footprints:
WRS = fiona.open(basepath+'WRS-2_bound_world_0.kml', driver='KML')

In [10]:
#create lists to hold the paths and rows and BoxIDs
paths = []; rows = []; boxes = []

#loop through all features (path, row footprints)
for feature in WRS:
    # create shapely polygons from the Landsat footprints
    coordinates = feature['geometry']['coordinates'][0]
    coords = [xy[0:2] for xy in coordinates]
    pathrow_poly = Polygon(coords)
    
    # grab the path and row name from the WRS kml file:
    pathrowname = feature['properties']['Name']  
    path = pathrowname.split('_')[0]; row = pathrowname.split('_')[1]
#     print(path, row)
    
    #for each feature, loop through each of the box_points
    for BoxID in box_points:  
        box_points_in = 0 # counter for number of box_points in the pathrow_geom:
        points = box_points.get(BoxID) # grab the points corresponding to the ID
        for i in range(0, len(points)):
            point = points[i]
            # if the pathrow shape contains the point
            if point.within(pathrow_poly):
                box_points_in = box_points_in+1 # append the counter
        if box_points_in == 5: # if all box vertices are inside the footprint, save the path, row, BoxID
            paths.append('%03d' % int(path))
            rows.append('%03d' % int(row))
            boxes.append(BoxID)

# Store in dataframe
boxes_pr_df = pd.DataFrame(list(zip(boxes, paths, rows)), columns=['BoxID','Path', 'Row'])
boxes_pr_df = boxes_pr_df.sort_values(by='BoxID')
boxes_pr_df # display

,BoxID,Path,Row
0,008,031,006
1,008,030,006
2,008,029,006
3,008,028,006
4,008,027,006
5,008,032,005
6,008,031,005


In [11]:
#####################################################################################
PR_FILENAME = 'LS_pathrows_test.csv' # change the filename as desired
#####################################################################################
boxes_pr_df.to_csv(path_or_buf = basepath+PR_FILENAME, sep=',') # write to csv

# 3) Download Landsat metadata files from AWS for overlapping scenes
The old aws syntax for grabbing an individual Landsat 8 metadata file is as follows:

     aws --no-sign-request s3 cp s3://landsat-pds/c1/L8/path/row/LC08_XXXX_pathrow_yyyyMMdd_01_T1/LC08_XXXX_pathrow_yyyyMMdd_01_T1_MTL.txt /path_to/output/
     
The aws syntax for listing the contents within the aws bucket is as follows:

    aws s3 ls --request-payer requester s3://usgs-landsat/collection02/level-2/standard/oli-tirs/yyyy/path/row/LC08_LS2R_pathrow_yyyyMMdd_yyyyMMdd_02_T1/ 
    
If you use the old syntax for grabbing a metadata file, you will encounter an AccessDenied return. However, s3 is a valid argument command for the aws CLI. Writing this combined with ls (lists the files within a folder) and --request-payer requester (indicates the requester will be charged for data download) will allow you to access files within the USGS Landsat imagery bucket.

We can use the paths and rows in the dataframe to access the full Landsat 8 scene list and the corresponding metdata files. Read https://docs.opendata.aws/landsat-pds/readme.html to learn more.

The old way to download the metadatafiles into Path_Row folders was by using:

    aws --no-sign-request s3 cp s3://landsat-pds/c1/L8/031/005/ Output/path/LS8aws/Path031_Row005/ --recursive --exclude "*" --include "*MTL.txt" 
    
The new way to download the metadatafiles into Path_Row folders is by using:
    
    ! aws s3api get-object --bucket usgs-landsat --key collection02/level-2/standard/oli-tirs/yyyy/path/row/LC08_L2SP_pathrow_yyyyMMdd_yyyyMMdd_02_T1/LC08_L2SP_pathrow_yyyyMMdd_yyyyMMdd_02_T1_MTL.txt  --request-payer requester LC08_L2SP_pathrow_yyyyMMdd_yyyyMMdd_02_T1_MTL.txt

Including the --request-payer requester as part of this line indicates that the referenced user will be charged for data download.

In [12]:
# Read in csv file from Step 2
boxes_pr_df = pd.read_csv(basepath+PR_FILENAME, dtype=str)
boxes_pr_df = boxes_pr_df.set_index('BoxID'); boxes_pr_df

,Unnamed: 0,Path,Row
BoxID,,,
008,0,031,006
008,1,030,006
008,2,029,006
008,3,028,006
008,4,027,006
008,5,032,005
008,6,031,005


In [13]:
%env AWS_PROFILE=terminusmapping
!aws sts get-caller-identity

env: AWS_PROFILE=terminusmapping
{
    "UserId": "829804448335",
    "Account": "829804448335",
    "Arn": "arn:aws:iam::829804448335:root"
}


In [21]:
#years = ['2013','2014','2015','2016','2017','2018','2019','2020','2021']
# Loop through the dataframe containing overlapping path, row info:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']
    folder_name = 'Path'+p+'_Row'+r+'_c1' # folder name
    
    #collection = 'collection02/level-2/standard/oli-tirs/2021/'
    #totalp_in = bp_in+p+'/'+r+'/' # adds path and row folders to the path
        #image_path = 'LC08_L2SP_031006_20210326_20210402_02_T1/' # troubleshooting
        #textfile_path = 'LC08_L2SP_031006_20210326_20210402_02_T1_MTL.txt' # troubleshooting
    bp_out = downloadpath+folder_name+'/' # output path for the downloaded files
    
    # create Path_Row folders if they don't exist already
    if os.path.exists(bp_out):
        print(folder_name, " EXISTS ALREADY. SKIP.")
    else:
        os.mkdir(bp_out)
        print(folder_name+" directory made")
        
        for year in years:
            
            # grab list of images for the year
            
            find_imgs = 'aws s3 ls --request-payer requester s3://usgs-landsat/collection02/level-1/standard/oli-tirs/'
            find_imgs += year+'/'
            find_imgs += p+'/'+r+'/'
            

            result = subprocess.check_output(find_imgs,shell=True)
            results = result.split() # split string
            
            imagenames = []
            for line in results: # loop through strings
                line = str(line)
                if 'LC' in line: # find just the image names
                    imgname = line[2:-2]
                    imagenames.append(imgname)
            
            for imgname in imagenames:
                command = 'aws s3api get-object --bucket usgs-landsat --key collection02/level-1/standard/oli-tirs/'
                command += year+'/'
                command += p+'/'+r+'/'
                command += imgname+'/'+imgname+'_MTL.txt'
                command += ' --request-payer requester '
                command += bp_out+imgname+'_MTL.txt'
                print(command)
                
                subprocess.call(command,shell=True)
       
        # Old way to obtain files from AWS
#         construct command to download all metadata files
        #command = 'aws --no-sign-request s3 cp '+totalp_in+' '+bp_out+' --recursive --exclude "*" --include "*MTL.txt"'
      

Path031_Row006_c1 directory made
aws s3api get-object --bucket usgs-landsat --key collection02/level-1/standard/oli-tirs/2013/031/006/LC08_L1GT_031006_20130912_20200912_02_T2/LC08_L1GT_031006_20130912_20200912_02_T2_MTL.txt --request-payer requester /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row006_c1/LC08_L1GT_031006_20130912_20200912_02_T2_MTL.txt
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-09-13T10:38:17+00:00",
    "ContentLength": 12100,
    "ETag": "\"8ce6dce7c0ca4ec3af0aa318b33f8738\"",
    "VersionId": "UnEKi7RYfV1Bewqa6tJS_OdP6Kw2oerF",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bucket usgs-landsat --key collection02/level-1/standard/oli-tirs/2013/031/006/LC08_L1TP_031006_20130402_20200912_02_T2/LC08_L1TP_031006_20130402_20200912_02_T2_MTL.txt --request-payer requester /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row006_c1/LC08_L1TP_031006_20130402_202

{
    "AcceptRanges": "bytes",
    "LastModified": "2020-09-11T11:58:51+00:00",
    "ContentLength": 12277,
    "ETag": "\"692b0363a040b358c923458ad059d030\"",
    "VersionId": "Aj6yrSU8dVTSnUMSb4GNMmGfhZhKqrTR",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bucket usgs-landsat --key collection02/level-1/standard/oli-tirs/2014/031/006/LC08_L1TP_031006_20140915_20200911_02_T1/LC08_L1TP_031006_20140915_20200911_02_T1_MTL.txt --request-payer requester /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row006_c1/LC08_L1TP_031006_20140915_20200911_02_T1_MTL.txt
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-09-11T07:44:32+00:00",
    "ContentLength": 12279,
    "ETag": "\"2fea4878d74cc50393238d461eda0fab\"",
    "VersionId": "mrCwbfigU9hu.UphJmyHnmHkrH56EViC",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bu

{
    "AcceptRanges": "bytes",
    "LastModified": "2020-09-24T15:19:30+00:00",
    "ContentLength": 12278,
    "ETag": "\"3776676032b100aeb057b32e3a191680\"",
    "VersionId": "UpcRq1ju8qzTKmU4o7RxdhDgie_9N14v",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bucket usgs-landsat --key collection02/level-1/standard/oli-tirs/2015/031/006/LC08_L1TP_031006_20150902_20200908_02_T1/LC08_L1TP_031006_20150902_20200908_02_T1_MTL.txt --request-payer requester /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row006_c1/LC08_L1TP_031006_20150902_20200908_02_T1_MTL.txt
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-09-23T14:02:02+00:00",
    "ContentLength": 12278,
    "ETag": "\"3f29f0e1e29588a4998811c783f4eeca\"",
    "VersionId": "13vvfwoKCqjx.hXvdxHAWexp2MQpzOrY",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bu

{
    "AcceptRanges": "bytes",
    "LastModified": "2020-10-16T05:29:35+00:00",
    "ContentLength": 12279,
    "ETag": "\"7290fdca230f7db2ebc52b614fc5f5cd\"",
    "VersionId": "43BsMU92fJDEGRTYVasNlmf6V63QCMMM",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bucket usgs-landsat --key collection02/level-1/standard/oli-tirs/2016/031/006/LC08_L1TP_031006_20160702_20200906_02_T1/LC08_L1TP_031006_20160702_20200906_02_T1_MTL.txt --request-payer requester /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row006_c1/LC08_L1TP_031006_20160702_20200906_02_T1_MTL.txt
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-09-25T03:06:56+00:00",
    "ContentLength": 12279,
    "ETag": "\"1cb36d8920953f92198d377e445d122f\"",
    "VersionId": "_oHZLd0ISaDgpNdDA4BLUC2QakknuEh4",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bu

{
    "AcceptRanges": "bytes",
    "LastModified": "2020-09-23T02:03:58+00:00",
    "ContentLength": 12279,
    "ETag": "\"621337d87d7c71c1247a034aa94d8e30\"",
    "VersionId": "YOE9bxWFNEqa6RlRytuY.bvt9GzD9ynQ",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bucket usgs-landsat --key collection02/level-1/standard/oli-tirs/2017/031/006/LC08_L1TP_031006_20170603_20201016_02_T2/LC08_L1TP_031006_20170603_20201016_02_T2_MTL.txt --request-payer requester /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row006_c1/LC08_L1TP_031006_20170603_20201016_02_T2_MTL.txt
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-10-16T05:36:38+00:00",
    "ContentLength": 12280,
    "ETag": "\"1651d986c89228410d2d2e91785a8b1e\"",
    "VersionId": "26IDgtopwAQbrs5u2IQJyCtIsPx6R94u",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bu

...

{
    "AcceptRanges": "bytes",
    "LastModified": "2020-10-16T05:37:25+00:00",
    "ContentLength": 12350,
    "ETag": "\"fa58a78e87e0cdec21577ee626799760\"",
    "VersionId": "4Pc1iEA9vimc6Y6Lw0vc6bJ95BDXwa2c",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bucket usgs-landsat --key collection02/level-1/standard/oli-tirs/2017/031/005/LC08_L1TP_031005_20170923_20200903_02_T1/LC08_L1TP_031005_20170923_20200903_02_T1_MTL.txt --request-payer requester /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row005_c1/LC08_L1TP_031005_20170923_20200903_02_T1_MTL.txt
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-09-03T02:00:38+00:00",
    "ContentLength": 12351,
    "ETag": "\"026aa02713398639cfb973977f4574e5\"",
    "VersionId": "KnLBiSe5yDlOzhUMeMosChCeBSqxQlBK",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bu

{
    "AcceptRanges": "bytes",
    "LastModified": "2020-08-31T02:20:54+00:00",
    "ContentLength": 12350,
    "ETag": "\"7c031d22a90146e676c8cc207f1afc1d\"",
    "VersionId": "NhQJ4y9RiCzlU_rlGj9BTH6mWoYLAcXI",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bucket usgs-landsat --key collection02/level-1/standard/oli-tirs/2018/031/005/LC08_L1TP_031005_20180910_20200830_02_T1/LC08_L1TP_031005_20180910_20200830_02_T1_MTL.txt --request-payer requester /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row005_c1/LC08_L1TP_031005_20180910_20200830_02_T1_MTL.txt
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-08-30T22:26:39+00:00",
    "ContentLength": 12279,
    "ETag": "\"3e9ee4d99b1bd3c51c71db8fc2d1e82f\"",
    "VersionId": "Yph.FNcIVLjhLsBgL5bd8bT0GSm4IK0e",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bu

{
    "AcceptRanges": "bytes",
    "LastModified": "2020-08-29T14:08:51+00:00",
    "ContentLength": 12352,
    "ETag": "\"af411389b6ff4f424ed5f2063091f9db\"",
    "VersionId": "O4lKJXDm0WM.uvHy1f5fVz8G.HqtLwP3",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bucket usgs-landsat --key collection02/level-1/standard/oli-tirs/2019/031/005/LC08_L1TP_031005_20190727_20200828_02_T1/LC08_L1TP_031005_20190727_20200828_02_T1_MTL.txt --request-payer requester /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row005_c1/LC08_L1TP_031005_20190727_20200828_02_T1_MTL.txt
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-08-28T15:56:48+00:00",
    "ContentLength": 12350,
    "ETag": "\"09e82e69437169803e68f03eb5d55b8e\"",
    "VersionId": "zd4dYmHXCGop9Hr7K7onirywVLSxuwi3",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bu

{
    "AcceptRanges": "bytes",
    "LastModified": "2020-12-05T08:39:45+00:00",
    "ContentLength": 12366,
    "ETag": "\"16948a148becee3fb7c62fc0345e2b20\"",
    "VersionId": "eDPDAaRNpmhB1Q1gXicnx0cIvGRjMg6.",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bucket usgs-landsat --key collection02/level-1/standard/oli-tirs/2020/031/005/LC08_L1TP_031005_20200814_20200920_02_T1/LC08_L1TP_031005_20200814_20200920_02_T1_MTL.txt --request-payer requester /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row005_c1/LC08_L1TP_031005_20200814_20200920_02_T1_MTL.txt
{
    "AcceptRanges": "bytes",
    "LastModified": "2020-10-30T23:27:54+00:00",
    "ContentLength": 12295,
    "ETag": "\"47186b8b0aacf5279f96ea9f115ad53c\"",
    "VersionId": "xsxvQBszx20Ot5v9Jh_HAUDtMXq3pDxx",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bu

{
    "AcceptRanges": "bytes",
    "LastModified": "2021-06-08T04:03:52+00:00",
    "ContentLength": 12368,
    "ETag": "\"70038fc170c0328b4a7e5cf9ae93a8a9\"",
    "VersionId": "ojFUd.DdbioSOkIn_xZWxuQbFHJ9JgsC",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bucket usgs-landsat --key collection02/level-1/standard/oli-tirs/2021/031/005/LC08_L1TP_031005_20210614_20210622_02_T1/LC08_L1TP_031005_20210614_20210622_02_T1_MTL.txt --request-payer requester /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row005_c1/LC08_L1TP_031005_20210614_20210622_02_T1_MTL.txt
{
    "AcceptRanges": "bytes",
    "LastModified": "2021-06-22T12:38:37+00:00",
    "ContentLength": 12366,
    "ETag": "\"2e199822d66287306d9822473bf5f908\"",
    "VersionId": "agTQguZczF_hn7EO5oJ0WtFaLxRPaSL6",
    "ContentType": "text/plain",
    "Metadata": {},
    "RequestCharged": "requester",
    "TagCount": 3
}
aws s3api get-object --bu

# 4) Calculate cloud % over terminus box using Landsat quality band

If the terminus box shapefiles were not originally in UTM projection, will need to reproject them into UTM to match the Landsat 8 projection. The code automatically finds the UTM zones from the metadata files and fills in the following syntax to reproject:
    
    ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:326zone -s_srs EPSG:3413 output.shp input.shp
    
If they are already in UTM projection, skip this step and rename the files to end with "\_UTM\_##.shp" where ## corresponds to the zone number (e.g., "\_UTM\_07.shp", "\_UTM\_21.shp").

In [22]:
zones = {} # initialize dictionary to hold UTM zone for each Landsat scene path row
zone_list = [] # list of zones

# Loop through all scenes:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']
    BoxID = str(index)
    folder_name = 'Path'+p+'_Row'+r+'_c1'
    pr_folderpath = downloadpath+folder_name+'/' # path to the downloaded metadata files
    pathtoshp = basepath+"Box"+BoxID+"/Box"+BoxID # path to the terminus box shapefiles (all projections)
    
    if len(os.listdir(pr_folderpath)) > 0: # if there are files in the folder
        # grab UTM Zone from the first metadata file
        #mtl_scene = os.listdir(pr_folderpath)[0]
        mtl_scene = glob.glob(pr_folderpath+'*_MTL.txt')[0]
        print(mtl_scene)
        mtl = open(mtl_scene, 'r')
        #mtl = open(pr_folderpath+mtl_scene+'/'+mtl_scene+'_MTL.txt', 'r')
#         mtl = open(pr_folderpath+'/'+'LC08_L2SP_031006_20210326_20210402_02_T1_MTL.txt')
        zone_found = 0
        for line in mtl:  #loop through lines in metadata to find the UTM ZONE
            variable = line.split("=")[0]
            if ("UTM_ZONE" in variable) and zone_found == 0:
                #save it:
                zone = '%02d' % int(line.split("=")[1][1:-1])
                zones.update({folder_name: zone}); zone_list.append(zone)
                zone_found = 1
                
        # reproject shapefile(s) into UTM
        zone = zones[folder_name]
        rp_shp = 'ogr2ogr -f "ESRI Shapefile" '+pathtoshp+'_UTM_'+zone+'.shp '+pathtoshp+'_WGS.shp -t_srs EPSG:326'+zone+' -s_srs EPSG:4326'
        print(rp_shp)
        subprocess.call(rp_shp, shell=True)
        
    else: # if no files in folder, zone = nan, must fill in manually
        zone_list.append(np.nan)
        
boxes_pr_df['Zone'] = zone_list # add to the path row dataframe
print("Done.")

/Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row006_c1/LC08_L1TP_031006_20200510_20200820_02_T2_MTL.txt
ogr2ogr -f "ESRI Shapefile" /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_UTM_19.shp /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_WGS.shp -t_srs EPSG:32619 -s_srs EPSG:4326
/Users/phoebekinzelman/research/greenland/LS8aws/Path030_Row006_c1/LC08_L1TP_030006_20200706_20200913_02_T1_MTL.txt
ogr2ogr -f "ESRI Shapefile" /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_UTM_19.shp /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_WGS.shp -t_srs EPSG:32619 -s_srs EPSG:4326
/Users/phoebekinzelman/research/greenland/LS8aws/Path029_Row006_c1/LC08_L1TP_029006_20170504_20200904_02_T2_MTL.txt
ogr2ogr -f "ESRI Shapefile" /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_UTM_19.shp /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_WGS.shp -t_srs EPSG:32619 -s_srs EP

In [23]:
# resave path row csv file with UTM zone information, see above for variable PR_FILENAME
boxes_pr_df.to_csv(path_or_buf = basepath+PR_FILENAME, sep=',')

Use gdalwarp commands and the __vsicurl__ link to download subset of the quality band we will use to determine cloud cover:

    gdalwarp -cutline path_to_shp.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/031/005/imagename/imagename_QA_PIXEL.TIF path_to_subsetQAPIXEL.TIF
    
These subset BQA files will go into the generated BoxID folders.

In [53]:
# Loop through all scenes:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']
    zone = row['Zone'] # new zone data
    BoxID = str(index)
    folder_name = 'Path'+p+'_Row'+r+'_c1'
    pr_folderpath = downloadpath+folder_name+'/' # path to the downloaded metadata files
    pathtoshp = basepath+"Box"+BoxID+"/Box"+BoxID # path to the terminus box shapefiles (all projections)

    files = os.listdir(pr_folderpath) # grab the names of the LS8 scenes

    for file in files:
        scene = file[:-8]
#         if len(scenes) > 0 and scene.startswith("LC08") and 'L1TP' in scene: # L1TP scenes
        if len(scene) > 0 and scene.startswith("LC") and 'L1TP' in scene and 'T1' in file: # L1TP scenes
            pathtoshp_rp = pathtoshp+'_UTM_'+zone # path to the UTM box shapefile
            for year in years:
                if scene[17:21] != year:
                    print(year, scene, scene[17:21])
                    continue
                print("MADE IT")
                #pathtoQAPIXEL = '/vsis3/https://usgs-landsat.s3.amazonaws.com/collection02/level-2/standard/oli-tirs/'
                pathtoQAPIXEL = '/vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/'
                pathtoQAPIXEL += year+'/'
                pathtoQAPIXEL += p+'/'+r+'/'
                #set path to the quality band (QA_PIXEL.TIF) in the cloud using the scene name
                #(old line of code using vsicurl): pathtoQAPIXEL = '/vsicurl/s3://usgs-landsat/collection02/level-2/standard/oli-tirs/'+p+'/'+r+'/'+scene+"/"+scene+"_QA_PIXEL.TIF"
                pathtoQAPIXEL += scene+'/'+scene+"_QA_PIXEL.TIF"
                subsetout = pr_folderpath+scene+'_QAPIXEL_Box'+BoxID+'.TIF' # output file naming

                # construct download command
                QAPIXEL_dwnld_cmd='gdalwarp -overwrite -cutline '+pathtoshp_rp+'.shp -crop_to_cutline '+pathtoQAPIXEL+' '+subsetout+ ' --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2 '#--config AWS_SECRET_ACCESS_KEY '+SECRET_KEY + ' --config AWS_ACCESS_KEY_ID '+ ACCESS_KEY
                print(QAPIXEL_dwnld_cmd) # check command syntax before downloading

                # Uncomment the following to run:
                subprocess.call(QAPIXEL_dwnld_cmd, shell=True)
                print(scene+'_QAPIXEL_Box'+BoxID+'.TIF subset downloaded')

2013 LC08_L1TP_031006_20150427_20200909_02_T1 2015
2014 LC08_L1TP_031006_20150427_20200909_02_T1 2015
MADE IT
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2015/031/006/LC08_L1TP_031006_20150427_20200909_02_T1/LC08_L1TP_031006_20150427_20200909_02_T1_QA_PIXEL.TIF /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row006_c1/LC08_L1TP_031006_20150427_20200909_02_T1_QAPIXEL_Box008.TIF --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2 
Creating output file that is 543P x 254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2015/031/006/LC08_L1TP_031006_20150427_20200909_02_T1/LC08_L1TP_031006_20150427_20200909_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031006_20150427_20200909_02_T1_QAPIXEL_Box008.TIF subset downloaded
2016 LC08_L1TP_031006_20150427_2020

Creating output file that is 543P x 254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/031/006/LC08_L1TP_031006_20180910_20200830_02_T1/LC08_L1TP_031006_20180910_20200830_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031006_20180910_20200830_02_T1_QAPIXEL_Box008.TIF subset downloaded
2019 LC08_L1TP_031006_20180910_20200830_02_T1 2018
2020 LC08_L1TP_031006_20180910_20200830_02_T1 2018
2021 LC08_L1TP_031006_20180910_20200830_02_T1 2018
2013 LC08_L1TP_031006_20150411_20200909_02_T1 2015
2014 LC08_L1TP_031006_20150411_20200909_02_T1 2015
MADE IT
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2015/031/006/LC08_L1TP_031006_20150411_20200909_02_T1/LC08_L1TP_031006_20150411_20200909_02_T1_QA_PIXEL.TIF /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row006_c1/LC08_L1TP_

Creating output file that is 543P x 254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/031/006/LC08_L1TP_031006_20180403_20200901_02_T1/LC08_L1TP_031006_20180403_20200901_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031006_20180403_20200901_02_T1_QAPIXEL_Box008.TIF subset downloaded
2019 LC08_L1TP_031006_20180403_20200901_02_T1 2018
2020 LC08_L1TP_031006_20180403_20200901_02_T1 2018
2021 LC08_L1TP_031006_20180403_20200901_02_T1 2018
2013 LC08_L1TP_031006_20180505_20200901_02_T1 2018
2014 LC08_L1TP_031006_20180505_20200901_02_T1 2018
2015 LC08_L1TP_031006_20180505_20200901_02_T1 2018
2016 LC08_L1TP_031006_20180505_20200901_02_T1 2018
2017 LC08_L1TP_031006_20180505_20200901_02_T1 2018
MADE IT
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/031/006/LC08_L1TP_031006_20

Creating output file that is 543P x 254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2016/031/006/LC08_L1TP_031006_20160312_20200907_02_T1/LC08_L1TP_031006_20160312_20200907_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031006_20160312_20200907_02_T1_QAPIXEL_Box008.TIF subset downloaded
2017 LC08_L1TP_031006_20160312_20200907_02_T1 2016
2018 LC08_L1TP_031006_20160312_20200907_02_T1 2016
2019 LC08_L1TP_031006_20160312_20200907_02_T1 2016
2020 LC08_L1TP_031006_20160312_20200907_02_T1 2016
2021 LC08_L1TP_031006_20160312_20200907_02_T1 2016
2013 LC08_L1TP_031006_20200713_20200912_02_T1 2020
2014 LC08_L1TP_031006_20200713_20200912_02_T1 2020
2015 LC08_L1TP_031006_20200713_20200912_02_T1 2020
2016 LC08_L1TP_031006_20200713_20200912_02_T1 2020
2017 LC08_L1TP_031006_20200713_20200912_02_T1 2020
2018 LC08_L1TP_031006_20200713_20200912_02_T1 2020
2019 LC08_L1TP_031006_20200713_20200912_02_T1 2020
MADE IT
gdalwarp -ove

Creating output file that is 543P x 254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/031/006/LC08_L1TP_031006_20190711_20200828_02_T1/LC08_L1TP_031006_20190711_20200828_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031006_20190711_20200828_02_T1_QAPIXEL_Box008.TIF subset downloaded
2020 LC08_L1TP_031006_20190711_20200828_02_T1 2019
2021 LC08_L1TP_031006_20190711_20200828_02_T1 2019
2013 LC08_L1TP_031006_20150716_20200908_02_T1 2015
2014 LC08_L1TP_031006_20150716_20200908_02_T1 2015
MADE IT
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2015/031/006/LC08_L1TP_031006_20150716_20200908_02_T1/LC08_L1TP_031006_20150716_20200908_02_T1_QA_PIXEL.TIF /Users/phoebekinzelman/research/greenland/LS8aws/Path031_Row006_c1/LC08_L1TP_031006_20150716_20200908_02_T1_QAPIXEL_Box008.TIF -

...

Creating output file that is 543P x 254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/031/005/LC08_L1TP_031005_20190609_20200828_02_T1/LC08_L1TP_031005_20190609_20200828_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031005_20190609_20200828_02_T1_QAPIXEL_Box008.TIF subset downloaded
2020 LC08_L1TP_031005_20190609_20200828_02_T1 2019
2021 LC08_L1TP_031005_20190609_20200828_02_T1 2019
2013 LC08_L1TP_031005_20180708_20200831_02_T1 2018
2014 LC08_L1TP_031005_20180708_20200831_02_T1 2018
2015 LC08_L1TP_031005_20180708_20200831_02_T1 2018
2016 LC08_L1TP_031005_20180708_20200831_02_T1 2018
2017 LC08_L1TP_031005_20180708_20200831_02_T1 2018
MADE IT
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/031/005/LC08_L1TP_031005_20180708_20200831_02_T1/LC08_L1TP_031005_20180708_202

Creating output file that is 543P x 254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2015/031/005/LC08_L1TP_031005_20150630_20201016_02_T1/LC08_L1TP_031005_20150630_20201016_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031005_20150630_20201016_02_T1_QAPIXEL_Box008.TIF subset downloaded
2016 LC08_L1TP_031005_20150630_20201016_02_T1 2015
2017 LC08_L1TP_031005_20150630_20201016_02_T1 2015
2018 LC08_L1TP_031005_20150630_20201016_02_T1 2015
2019 LC08_L1TP_031005_20150630_20201016_02_T1 2015
2020 LC08_L1TP_031005_20150630_20201016_02_T1 2015
2021 LC08_L1TP_031005_20150630_20201016_02_T1 2015
2013 LC08_L1TP_031005_20190422_20200828_02_T1 2019
2014 LC08_L1TP_031005_20190422_20200828_02_T1 2019
2015 LC08_L1TP_031005_20190422_20200828_02_T1 2019
2016 LC08_L1TP_031005_20190422_20200828_02_T1 2019
2017 LC08_L1TP_031005_20190422_20200828_02_T1 2019
2018 LC08_L1TP_031005_20190422_20200828_02_T1 2019
MADE IT
gdalwarp -ove

Creating output file that is 543P x 254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2014/031/005/LC08_L1TP_031005_20140915_20200911_02_T1/LC08_L1TP_031005_20140915_20200911_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031005_20140915_20200911_02_T1_QAPIXEL_Box008.TIF subset downloaded
2015 LC08_L1TP_031005_20140915_20200911_02_T1 2014
2016 LC08_L1TP_031005_20140915_20200911_02_T1 2014
2017 LC08_L1TP_031005_20140915_20200911_02_T1 2014
2018 LC08_L1TP_031005_20140915_20200911_02_T1 2014
2019 LC08_L1TP_031005_20140915_20200911_02_T1 2014
2020 LC08_L1TP_031005_20140915_20200911_02_T1 2014
2021 LC08_L1TP_031005_20140915_20200911_02_T1 2014
2013 LC08_L1TP_031005_20150411_20201016_02_T1 2015
2014 LC08_L1TP_031005_20150411_20201016_02_T1 2015
MADE IT
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1

Creating output file that is 543P x 254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/031/005/LC08_L1TP_031005_20180825_20200831_02_T1/LC08_L1TP_031005_20180825_20200831_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031005_20180825_20200831_02_T1_QAPIXEL_Box008.TIF subset downloaded
2019 LC08_L1TP_031005_20180825_20200831_02_T1 2018
2020 LC08_L1TP_031005_20180825_20200831_02_T1 2018
2021 LC08_L1TP_031005_20180825_20200831_02_T1 2018
2013 LC08_L1TP_031005_20170416_20200904_02_T1 2017
2014 LC08_L1TP_031005_20170416_20200904_02_T1 2017
2015 LC08_L1TP_031005_20170416_20200904_02_T1 2017
2016 LC08_L1TP_031005_20170416_20200904_02_T1 2017
MADE IT
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2017/031/005/LC08_L1TP_031005_20170416_20200904_02_T1/LC08_L1TP_031005_20170416_202

Creating output file that is 543P x 254L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2016/031/005/LC08_L1TP_031005_20160413_20200907_02_T1/LC08_L1TP_031005_20160413_20200907_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031005_20160413_20200907_02_T1_QAPIXEL_Box008.TIF subset downloaded
2017 LC08_L1TP_031005_20160413_20200907_02_T1 2016
2018 LC08_L1TP_031005_20160413_20200907_02_T1 2016
2019 LC08_L1TP_031005_20160413_20200907_02_T1 2016
2020 LC08_L1TP_031005_20160413_20200907_02_T1 2016
2021 LC08_L1TP_031005_20160413_20200907_02_T1 2016
2013 LC08_L1TP_031005_20160515_20200907_02_T1 2016
2014 LC08_L1TP_031005_20160515_20200907_02_T1 2016
2015 LC08_L1TP_031005_20160515_20200907_02_T1 2016
MADE IT
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2016/031/005/LC08_L1TP_031005_20

# 5) Create buffer zone around terminus boxes and rasterize terminus boxes

In [54]:
buffers = []
mindimensions = []

# Calculate a buffer distance around the terminus box:
for BoxID in BoxIDs:
    for file in os.listdir(basepath+'Box'+BoxID+'/'):
        if 'UTM' in file and '.shp' in file and "Box" in file: # identify UTM projected box
            boxpath = basepath+"Box"+BoxID+"/"+file  
            termbox = fiona.open(boxpath)
            
    # grab the box coordinates:
    box = termbox.next(); box_geom= box.get('geometry'); box_coords = box_geom.get('coordinates')[0]
    points = []
    for coord_pair in box_coords:
        lat = coord_pair[0]; lon = coord_pair[1]; points.append([lat, lon])
            
    # Calculate distance between coord 1 and 2 and between 2 and 3
    coord1 = points[0]; coord2 = points[1]; coord3 = points[2]   
    dist1 = distance(coord1[0], coord1[1], coord2[0], coord2[1]);
    dist2 = distance(coord2[0], coord2[1], coord3[0], coord3[1]) 
    buff_dist = int(np.max([dist1, dist2])) # pick the longer one as the buffer distance
    mindim = int(np.min([dist1, dist2]))/15.0 # calculate the minimum dimension in pixels
    
    # record:
    buffers.append(buff_dist)
    mindimensions.append(int(mindim))

# store as dataframe:
buff_df = pd.DataFrame(list(zip(BoxIDs, buffers, mindimensions)), columns=['BoxID', 'Buff_dist_m', 'min_dim_px'])
buff_df

/var/folders/kw/nmc8p1t51rq1bkcn6xrr0yv40000gr/T/ipykernel_61071/618081620.py:12: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  box = termbox.next(); box_geom= box.get('geometry'); box_coords = box_geom.get('coordinates')[0]


,BoxID,Buff_dist_m,min_dim_px
0,008,15792,282


In [55]:
######################################################################################
BOX_FILENAME = 'Buffdist_test.csv'
buff_df.to_csv(basepath+BOX_FILENAME) # write to csv, change name as desired
######################################################################################

The next section creates a buffer zone shapefile using GDAL command **ogr2ogr** with the following syntax:

    ogr2ogr Buffer###.shp path_to_terminusbox###.shp  -dialect sqlite -sql "SELECT ST_Buffer(geometry, buffer_distance) AS geometry,*FROM 'Box###'" -f "ESRI Shapefile"

In [56]:
# loop through the buffer distance dataframe:
for index, row in buff_df.iterrows():
    BoxID = row['BoxID']
    buff_dist = str(row['Buff_dist_m'])
    
    terminusbox_path = basepath+"Box"+BoxID+"/Box"+BoxID+".shp" # path to box shapefile
    outputbuffer_path = basepath+"Box"+BoxID+"/Buffer"+BoxID+".shp" # path and name of new buffer file
    
    # Set buffer command
    buffer_cmd = 'ogr2ogr '+outputbuffer_path+" "+terminusbox_path+' -dialect sqlite -sql "SELECT ST_Buffer(geometry, '+buff_dist+") AS geometry,*FROM 'Box"+BoxID+"'"+'" -f "ESRI Shapefile"'
    print(buffer_cmd)
    
    subprocess.call(buffer_cmd, shell=True) # run on terminal

ogr2ogr /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008.shp /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008.shp -dialect sqlite -sql "SELECT ST_Buffer(geometry, 15792) AS geometry,*FROM 'Box008'" -f "ESRI Shapefile"


The terminus box shapefiles are then rasterized (to be used as a mask during the WTMM filering) using the GDAL **gdal_rasterize** command, subset to the buffer zone using the GDAL **gdalwarp** command, and reprojected:

1) Rasterize

        gdal_rasterize -burn 1.0 -tr x_resolution y_resolution -a_nodata 0.0 path_to_terminusbox.shp path_to_terminusbox_raster.TIF
    
2) Subset

        gdalwarp -cutline path_to_Buffer###.shp -crop_to_cutline path_to_terminusbox_raster.TIF path_to_subset_raster_cut.TIF
    
3) Reproject to UTM

In [57]:
######################################################################################
source_srs = '3413' # original projection of the shapefiles (Greenland polar stereo = 3413)
######################################################################################

for index, row in buff_df.iterrows():
    BoxID = row['BoxID']
    terminusbox_path = basepath+"Box"+BoxID+"/Box"+BoxID # path to box
    buffer_path = basepath+"Box"+BoxID+"/Buffer"+BoxID # path to buffer
    terminusraster_path = basepath+"Box"+BoxID+"/Box"+BoxID+".TIF" # path to rasterized box
    cutraster_path = basepath+"Box"+BoxID+"/Box"+BoxID+"_raster_cut.TIF" # name for cropped file
    
    zones = boxes_pr_df.loc[BoxID, 'Zone'] # grab zone matching BoxID from other dataframe
    # Set commands
    rasterize_cmd = 'gdal_rasterize -burn 1.0 -tr 15.0 15.0 -a_nodata 0.0 '+terminusbox_path+'.shp '+terminusraster_path
    subsetbuffer_cmd = 'gdalwarp -cutline '+buffer_path+'.shp -crop_to_cutline '+terminusraster_path+' '+cutraster_path
    subprocess.call(rasterize_cmd, shell=True) # rasterize with command terminal
    subprocess.call(subsetbuffer_cmd, shell=True) # subset to buffer with command terminal
    
    for zone in zones:
        
        # Set commands
        rp_shp = 'ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:326'+zone+' -s_srs EPSG:'+source_srs+' '+buffer_path+"_UTM_"+zone+".shp "+buffer_path+'.shp'
        print(rp_shp)
        print(rasterize_cmd)
        print(subsetbuffer_cmd)
        #subprocess.call(rp_shp, shell=True) # reproject

    print("Box"+BoxID) # check progress

0...10...20...30...40...50...60...70...80...90...100 - done.
ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:32619 -s_srs EPSG:3413 /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008_UTM_19.shp /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008.shp
gdal_rasterize -burn 1.0 -tr 15.0 15.0 -a_nodata 0.0 /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008.shp /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008.TIF
gdalwarp -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008.shp -crop_to_cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008.TIF /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_raster_cut.TIF
ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:32619 -s_srs EPSG:3413 /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008_UTM_19.shp /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008.shp
gdal_rasterize -burn 1.0 -tr

ERROR 1: Output dataset /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_raster_cut.TIF exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.



In [58]:
list(set(zone))

['9', '1']

# 6) Download non-cloudy Landsat images from AWS

To remove cloudy images, we will find the number of pixels in our terminus box that correspond to a cumulative pixel value of > 4096 in the QA_PIXEL band. If the fraction of cloudy pixels with values is above the threshold, we won't download the image. 

Additionally, we remove images that are primarily black (fill value of 0 or 1 in QA_PIXEL band). This ensures that the scenes that cut off halfway across the glacier are not included in further analysis. The fill percent threshold may need to be adjusted.

In [59]:
######################################################################################
# These are the recommended values. Adjust thresholds here:
QAPIXEL_thresh = 22280.0 # QA pixel value threshold to be considered cloud
cpercent_thresh = 50.0 # maximum cloud cover % in terminus box
fpercent_thresh = 60.0 # maximum fill % in terminus box

# Set your desired Landsat 8 bands to download:
bands = [8] # panchromatic
# bands = [2, 3, 4] # multispectral

######################################################################################

In [60]:
# Download images that pass these thresholds:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; zone = row['Zone']; r = row['Row']; BoxID = index; 
    folder_name = 'Path'+p+'_Row'+r+'_c1'
    pr_folderpath = downloadpath+folder_name+'/'
    
    bp_out = downloadpath+'Box'+BoxID+'/' # folder name for downloaded images
    if os.path.exists(bp_out): # create folder if it does not exist
        print("Box"+BoxID, " exists already. Skip creation of directory.")
    else:
        os.mkdir(bp_out)
        print("Box"+BoxID+" directory made.")
    
    # path to the shapefile covering the region that will be downloaded
    pathtobuffer = basepath+'Box'+BoxID+'/Buffer'+BoxID+'_UTM_'+zone+'.shp'  # buffer around box - recommended
#     pathtobox = basepath+'Box'+BoxID+'/Box'+BoxID+'_UTM_'+zone+'.shp' # just the box
    
    for scene in os.listdir(pr_folderpath):
        if len(os.listdir(pr_folderpath)) > 0 and scene.startswith("LC") and scene.endswith(".TIF") and 'T1' in scene: #may need to specify '_QAPIXEL_Box'+BoxID+'.TIF' for the endswith function
            scene = scene[:-19]
            year = scene[17:21] # grab acquisition year
            
            QApixelpath = pr_folderpath+scene+'_QAPIXEL_Box'+BoxID+'.TIF' #pr_folderpath+scene+"/"+scene+'_QA_Box'+BoxID+'.TIF' # path to QA_PIXEL file
            #pr_folderpath+scene+'_QAPIXEL_Box'+BoxID+'.TIF'
            subsetQApixel = mpimg.imread(QApixelpath) # read in as numpy array
            
            totalpixels = subsetQApixel.shape[0]*subsetQApixel.shape[1] # total number of pixels
            cloudQApixel = subsetQApixel[subsetQApixel > QAPIXEL_thresh] # cloudy pixels (value > QAPIXEL_thresh)
            fillQApixel = subsetQApixel[subsetQApixel < 2.0] # fill pixels (value = 0 or 1)
            cloudpixels = len(cloudQApixel); fillpixels = len(fillQApixel) # count the cloudy and fill pixels
            cloudpercent = int(float(cloudpixels)/float(totalpixels)*100) # calculate percent cloudy
            fillpercent = int(float(fillpixels)/float(totalpixels)*100) # calculate percent fill
            print(scene, 'Cloud % ', cloudpercent, 'Fill %', fillpercent) # check values
            
            # evaluate thresholds
            if cloudpercent <= cpercent_thresh and fillpercent <= fpercent_thresh:
                # download the bands for that scene into your scene folders:
                for band in bands:
                        band = str(band) # string format

                        # input path to your bands in AWS:
                        pathin = '/vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/'+year+'/'+p+"/"+r+"/"+scene+"/"+scene+"_B"+band+".TIF"
                        # output path to the overall BoxID folder
                        outfilename = scene[0:-19]+"_B"+band+'_Buffer'+BoxID+'.TIF'
                        pathout = downloadpath+'Box'+BoxID+'/'+outfilename

                        # construct download command
                        download_cmd = 'gdalwarp -overwrite -cutline '+pathtobuffer+' -crop_to_cutline '+pathin+' '+pathout+ ' --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2 '
    #                     download_cmd = 'gdalwarp -cutline '+pathtobox+' -crop_to_cutline '+pathin+' '+pathout
                                        #'gdalwarp -overwrite -cutline '+pathtoshp_rp+'.shp -crop_to_cutline '+pathtoQAPIXEL+' '+subsetout+ ' --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2 '#--config AWS_SECRET_ACCESS_KEY '+SECRET_KEY + ' --config AWS_ACCESS_KEY_ID '+ ACCESS_KEY

                        print(download_cmd) # check

                         # Once checked, uncomment the following to commence download:                   
                        subprocess.call(download_cmd, shell=True)
                        print(outfilename+" downloaded")

Box008  exists already. Skip creation of directory.
LC08_L1TP_031006_20200915_20200919_02_T1 Cloud %  17 Fill % 46
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/031/006/LC08_L1TP_031006_20200915_20200919_02_T1/LC08_L1TP_031006_20200915_20200919_02_T1_B8.TIF /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031006_2020_B8_Buffer008.TIF --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2 
Creating output file that is 3227P x 2647L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/031/006/LC08_L1TP_031006_20200915_20200919_02_T1/LC08_L1TP_031006_20200915_20200919_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2020/031/006/LC08_L1TP_031006_20200915_20200919_02_T1/LC08_L1TP_031006_20200915_20200919_02_T1_B8.TI

...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031006_2016_B8_Buffer008.TIF downloaded
LC08_L1TP_031006_20160429_20200907_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031006_20150529_20201015_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031006_20210326_20210402_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031006_20180505_20200901_02_T1 Cloud %  35 Fill % 46
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/031/006/LC08_L1TP_031006_20180505_20200901_02_T1/LC08_L1TP_031006_20180505_20200901_02_T1_B8.TIF /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031006_2018_B8_Buffer008.TIF --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2 
Creating output file that is 3227P x 2647L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/031/006/LC08_L1TP_031006_20180505_20200901_02_T1/LC08_L1TP_031006_

...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031006_2016_B8_Buffer008.TIF downloaded
LC08_L1TP_031006_20170502_20200904_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031006_20200424_20200822_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031006_20180910_20200830_02_T1 Cloud %  23 Fill % 46
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/031/006/LC08_L1TP_031006_20180910_20200830_02_T1/LC08_L1TP_031006_20180910_20200830_02_T1_B8.TIF /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031006_2018_B8_Buffer008.TIF --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2 
Creating output file that is 3227P x 2647L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/031/006/LC08_L1TP_031006_20180910_20200830_02_T1/LC08_L1TP_031006_20180910_20200830_02_T1_B8.TIF [1/1] : 0Using internal nodata v

...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031006_2020_B8_Buffer008.TIF downloaded
LC08_L1TP_031006_20210902_20210910_02_T1 Cloud %  26 Fill % 46
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/031/006/LC08_L1TP_031006_20210902_20210910_02_T1/LC08_L1TP_031006_20210902_20210910_02_T1_B8.TIF /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031006_2021_B8_Buffer008.TIF --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2 
Creating output file that is 3227P x 2647L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/031/006/LC08_L1TP_031006_20210902_20210910_02_T1/LC08_L1TP_031006_20210902_20210910_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/031/006/LC08_L1TP_031006_20210902_202

...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031006_2015_B8_Buffer008.TIF downloaded
LC08_L1TP_031006_20200307_20200822_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031006_20180708_20200831_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031006_20190625_20200829_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031006_20210310_20210317_02_T1 Cloud %  30 Fill % 46
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/031/006/LC08_L1TP_031006_20210310_20210317_02_T1/LC08_L1TP_031006_20210310_20210317_02_T1_B8.TIF /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031006_2021_B8_Buffer008.TIF --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2 
Creating output file that is 3227P x 2647L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2021/031/006/LC08_L1TP_031006_20210310_20210317_02_T1/LC08_L1TP_031006_

...

...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031005_2015_B8_Buffer008.TIF downloaded
LC08_L1TP_031005_20190913_20200827_02_T1 Cloud %  14 Fill % 46
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/031/005/LC08_L1TP_031005_20190913_20200827_02_T1/LC08_L1TP_031005_20190913_20200827_02_T1_B8.TIF /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031005_2019_B8_Buffer008.TIF --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2 
Creating output file that is 3227P x 2647L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/031/005/LC08_L1TP_031005_20190913_20200827_02_T1/LC08_L1TP_031005_20190913_20200827_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/031/005/LC08_L1TP_031005_20190913_202

...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031005_2019_B8_Buffer008.TIF downloaded
LC08_L1TP_031005_20180521_20200901_02_T1 Cloud %  0 Fill % 46
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/031/005/LC08_L1TP_031005_20180521_20200901_02_T1/LC08_L1TP_031005_20180521_20200901_02_T1_B8.TIF /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031005_2018_B8_Buffer008.TIF --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2 
Creating output file that is 3227P x 2647L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/031/005/LC08_L1TP_031005_20180521_20200901_02_T1/LC08_L1TP_031005_20180521_20200901_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/031/005/LC08_L1TP_031005_20180521_2020

...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031005_2015_B8_Buffer008.TIF downloaded
LC08_L1TP_031005_20190321_20200829_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031005_20150630_20201016_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031005_20160429_20200907_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031005_20210326_20210402_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031005_20200510_20200820_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031005_20140915_20200911_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031005_20181012_20200830_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031005_20161006_20200906_02_T1 Cloud %  20 Fill % 46
gdalwarp -overwrite -cutline /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Buffer008_UTM_19.shp -crop_to_cutline /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2016/031/005/LC08_L1TP_031005_20161006_20200906_02_T1/LC08_L1TP_031005_20161006_20200906_02_T1_B8.TIF /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031005_2016_B8_Buffer008.TIF -

Creating output file that is 3227P x 2647L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2017/031/005/LC08_L1TP_031005_20170822_20201016_02_T1/LC08_L1TP_031005_20170822_20201016_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2017/031/005/LC08_L1TP_031005_20170822_20201016_02_T1/LC08_L1TP_031005_20170822_20201016_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2017/031/005/LC08_L1TP_031005_20170822_20201016_02_T1/LC08_L1TP_031005_20170822_20201016_02_T1_B8.TIF to destination /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031005_2017_B8_Buffer008.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
LC08_L1TP_031005_2017_B8_Buffer008.TIF downloaded
LC08_L1TP_031005_20140408_20200911_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031005_20130402_20200912_02_T1 Cloud %  53 Fill % 46
LC08_L1TP_031005_202006

KeyboardInterrupt: 

### Reproject the downloaded files from UTM into your desired projection

In [61]:
# Reproject newly downloaded images:
for BoxID in list(set(boxes_pr_df.index)):
    bp_out = downloadpath+'Box'+BoxID+'/' # path to downloaded files
    
    # create output reprojected folder if does not exist
    if os.path.exists(bp_out+'reprojected/'):
        print("Box"+BoxID, "Reprojected EXISTS ALREADY.")
    else:
        os.mkdir(bp_out+'reprojected/')
        print("Box"+BoxID+" Reprojected directory made")
                      
    downloadedimages = os.listdir(bp_out) # all downloaded images
    for image in downloadedimages:
        if image.startswith("LC08"):
        #if image.startswith("LC05 / 7"):
            imagename = image[:-4] # remove suffix
            
            ######################################################################################
            # Gdalwarp command must be re-written for other projections.
            # The command and image name suffix is for Greenland Polar Stereo projection
            suffix = '_PS'
            rp_PS = "gdalwarp -t_srs EPSG:3413 "+bp_out+image+" "+bp_out+'reprojected/'+imagename+suffix+".TIF"
            subprocess.call(rp_PS, shell=True)   
            
            ######################################################################################
            
    print("Box"+BoxID+" reprojected.")

Box008 Reprojected EXISTS ALREADY.
Creating output file that is 4024P x 3729L.
Processing /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_028006_2014_B8_Buffer008.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_028006_2014_B8_Buffer008.TIF.
Copying nodata values from source /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_028006_2014_B8_Buffer008.TIF to destination /Users/phoebekinzelman/research/greenland/LS8aws/Box008/reprojected/LC08_L1TP_028006_2014_B8_Buffer008_PS.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 4024P x 3729L.
Processing /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_028006_2021_B8_Buffer008.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_028006_2021_B8_Buffer008.TIF.
Copying nodata values from source /Users/pho

...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 3919P x 3610L.
Processing /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_027006_2013_B8_Buffer008.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_027006_2013_B8_Buffer008.TIF.
Copying nodata values from source /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_027006_2013_B8_Buffer008.TIF to destination /Users/phoebekinzelman/research/greenland/LS8aws/Box008/reprojected/LC08_L1TP_027006_2013_B8_Buffer008_PS.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 4024P x 3729L.
Processing /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_028006_2015_B8_Buffer008.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_028006_2015_B8_Buffer008.TIF.
Copying nodata valu

...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 4024P x 3729L.
Processing /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031006_2018_B8_Buffer008.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031006_2018_B8_Buffer008.TIF.
Copying nodata values from source /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031006_2018_B8_Buffer008.TIF to destination /Users/phoebekinzelman/research/greenland/LS8aws/Box008/reprojected/LC08_L1TP_031006_2018_B8_Buffer008_PS.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 4024P x 3729L.
Processing /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_029006_2021_B8_Buffer008.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_029006_2021_B8_Buffer008.TIF.
Copying nodata valu

...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 4024P x 3729L.
Processing /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_030006_2020_B8_Buffer008.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_030006_2020_B8_Buffer008.TIF.
Copying nodata values from source /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_030006_2020_B8_Buffer008.TIF to destination /Users/phoebekinzelman/research/greenland/LS8aws/Box008/reprojected/LC08_L1TP_030006_2020_B8_Buffer008_PS.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 4024P x 3729L.
Processing /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_028006_2019_B8_Buffer008.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_028006_2019_B8_Buffer008.TIF.
Copying nodata valu

...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 4024P x 3729L.
Processing /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_030006_2014_B8_Buffer008.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_030006_2014_B8_Buffer008.TIF.
Copying nodata values from source /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_030006_2014_B8_Buffer008.TIF to destination /Users/phoebekinzelman/research/greenland/LS8aws/Box008/reprojected/LC08_L1TP_030006_2014_B8_Buffer008_PS.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 4024P x 3729L.
Processing /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031005_2017_B8_Buffer008.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/phoebekinzelman/research/greenland/LS8aws/Box008/LC08_L1TP_031005_2017_B8_Buffer008.TIF.
Copying nodata valu

In [62]:
os.listdir(downloadpath)

['Box378',
 'Box175',
 'Box347',
 'Box511',
 'Box181',
 'Box529',
 'Box186',
 'Box516',
 'Box340',
 'Box172',
 'Box349',
 'Box382',
 'Box376',
 'Box144',
 'Box520',
 'Box518',
 'Box188',
 'Box527',
 'Box143',
 'Box371',
 'Box385',
 'Box573',
 'Box325',
 'Box117',
 'Box587',
 'Box128',
 'Path028_Row006_c1',
 'Box580',
 'Box110',
 'Box322',
 'Box574',
 'Box542',
 'Box126',
 'Box314',
 'Box589',
 'Box119',
 'Box313',
 'Box121',
 'Box545',
 'Box189',
 'Box519',
 'Box384',
 'Box526',
 'Box370',
 'Box142',
 'Box348',
 'Box145',
 'Box377',
 'Box521',
 'Box383',
 'Box528',
 'Box517',
 'Box173',
 'Box341',
 'Box187',
 'Box379',
 'Box180',
 'Box346',
 'Box174',
 'Box510',
 'Box120',
 'Box312',
 'Box544',
 'Box543',
 'Box315',
 'Box127',
 'Box118',
 'Box588',
 'Box323',
 'Box111',
 'Box575',
 'Box581',
 'Box586',
 'Box572',
 'Box116',
 'Box324',
 'Path031_Row006_c1',
 'Box129',
 'Box269',
 'Box064',
 'Box256',
 'Box632',
 'Box400',
 'Box090',
 'Box438',
 'Box097',
 'Box407',
 'Box635',
 'Box251',

### Automatically grab the image acquisition dates from the metadata files

In [63]:
datetimes = [] # list of scene datetimes
scenes_dated = [] # list of scenes

# Loop through the dataframe with your path row combinations:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']; BoxID = index; 
    folder_name = 'Path'+p+'_Row'+r+'_c1'; print(folder_name)
    
    # Output folder paths"
    folderpath = downloadpath+folder_name+'/'
    bp_out = downloadpath+'Box'+BoxID+'/reprojected/'
      
    scenecount = 0 # keep track of the number of scenes:
    
    downloaded_scenes = os.listdir(bp_out)
    print(downloaded_scenes)
    for scene in downloaded_scenes:
        if scene.endswith('.TIF') and scene.startswith("LC08"):
            scenename = scene[:-20] # MAY NEED TO ADJUST DEPENDING ON SUFFIX - CHANGE THIS TO start and end
            print(scenename)
            if scenename in os.listdir(folderpath):
                scenefiles = os.listdir(folderpath+scenename+'/')
                for file in scenefiles:
                    if ("MTL.txt" in file): # open metadata file
                        mdata = open(folderpath+scenename+"/"+scenename+"_MTL.txt", "r")
                        for line in mdata:
                            variable = line.split("=")[0]
                            if ("DATE_ACQUIRED" in variable):
                                date = line.split("=")[1][1:-1] # find acquisition date

                        dates = datetime.datetime.strptime(date, '%Y-%m-%d') # save as datetime object
                        datetimes.append(dates); scenes_dated.append(scenename) # store in lists
                scenecount = scenecount+1

# Store in a dataframe
datetime_df = pd.DataFrame(list(zip(scenes_dated, datetimes)), columns=['Scene', 'datetime'])
datetime_df = datetime_df.sort_values(by='datetime', ascending=True); datetime_df = datetime_df.drop_duplicates()
datetime_df

Path031_Row006_c1
['LC08_L1TP_027006_2015_B8_Buffer008_PS.TIF', 'LC08_L1TP_028006_2013_B8_Buffer008_PS.TIF', 'LC08_L1TP_031006_2014_B8_Buffer008_PS.TIF', 'LC08_L1TP_031006_2015_B8_Buffer008_PS.TIF', 'LC08_L1TP_027006_2014_B8_Buffer008_PS.TIF', 'LC08_L1TP_031006_2016_B8_Buffer008_PS.TIF', 'LC08_L1TP_027006_2017_B8_Buffer008_PS.TIF', 'LC08_L1TP_029006_2018_B8_Buffer008_PS.TIF', 'LC08_L1TP_029006_2019_B8_Buffer008_PS.TIF', 'LC08_L1TP_027006_2016_B8_Buffer008_PS.TIF', 'LC08_L1TP_031006_2017_B8_Buffer008_PS.TIF', 'LC08_L1TP_030006_2018_B8_Buffer008_PS.TIF', 'LC08_L1TP_028006_2016_B8_Buffer008_PS.TIF', 'LC08_L1TP_028006_2017_B8_Buffer008_PS.TIF', 'LC08_L1TP_030006_2019_B8_Buffer008_PS.TIF', 'LC08_L1TP_028006_2014_B8_Buffer008_PS.TIF', 'LC08_L1TP_031006_2013_B8_Buffer008_PS.TIF', 'LC08_L1TP_032005_2018_B8_Buffer008_PS.TIF', 'LC08_L1TP_028006_2015_B8_Buffer008_PS.TIF', 'LC08_L1TP_027006_2013_B8_Buffer008_PS.TIF', 'LC08_L1TP_032005_2019_B8_Buffer008_PS.TIF', 'LC08_L1TP_032005_2014_B8_Buffer008_

,Scene,datetime


In [64]:
######################################################################################
DATES_FILENAME = 'imgdates_nonSE.csv' # change image date csv filename as desired
######################################################################################
datetime_df.to_csv(path_or_buf = basepath+DATES_FILENAME, sep=',') # write to csv

### Second pass at cloud filtering using range in pixel values

Removes cloudy images that slipped through QA filtering. Skip if unnecessary.

In [ ]:
# # convert all files in reprojected folder to png from TIF
# for BoxID in BoxIDs:
#     print(BoxID)
#     command = 'cd '+downloadpath+'Box'+BoxID+'/reprojected/; '+'mogrify -format png *_PS.TIF'
#     subprocess.call(command, shell=True)

In [ ]:
# ######################################################################################
# suffix = '_PS' # reprojection suffix
# ######################################################################################

# for BoxID in BoxIDs:
#     imagepath = downloadpath+'Box'+BoxID+'/reprojected/'
#     for img in os.listdir(imagepath):
#         if img.endswith('Buffer'+BoxID+suffix+'.png'):
#             image = cv2.imread(imagepath+img,-1) # read in image
#             imageplt = mpimg.imread(imagepath+img)
#             image_nofill = imageplt[imageplt > 0] # don't consider the fill points
#             img_std = np.std(image_nofill) # st. dev in values
#             if len(image_nofill.shape) > 1:
#                 img_range = np.max(image_nofill) - np.min(image_nofill)
#                 img_med = np.median(image_nofill)

#                 if img_std < 0.04 and img_med > 0.15: # adjust these threholds
#                     os.remove(imagepath+img) # remove png mimage
#                     os.remove(imagepath+img[:-4]+'.TIF') # remove pgm image     
#                     # show the image
#     #                 imgplt_trim = plt.imshow(cv2.cvtColor(imageplt, cv2.COLOR_BGR2RGB))
#     #                 plt.show()

### Grab fraction of total images available that were excluded due to clouds and fill

If you are interested in knowing how many images were filtered out using the cloud and fill thresholds, run the following cells. Otherwise, skip.

In [65]:
# read in path, row csv file if not already loaded
boxes_pr_df = pd.read_csv(basepath+PR_FILENAME, dtype=str)
boxes_pr_df = boxes_pr_df.set_index('BoxID'); BoxIDs = list(set(boxes_pr_df.index))
print(BoxIDs); boxes_pr_df.head()

['008']


,Unnamed: 0,Path,Row,Zone
BoxID,,,,
008,0,031,006,19
008,1,030,006,19
008,2,029,006,19
008,3,028,006,19
008,4,027,006,20


In [66]:
im_tots = []
downloaded = []
fractions = []

for BoxID in BoxIDs:
    pathrows_BoxID = boxes_pr_df[boxes_pr_df.index == BoxID].copy() # grab path rows for that BoxID
    
    im_tot = 0 # count number of total scenes available
    for idx, rw in pathrows_BoxID.iterrows():
        p = rw['Path']; r = rw['Row']
        ims_pr = len(os.listdir(downloadpath+'Path'+p+'_Row'+r+'_c1')) # grab number of scenes in that pathrow
        im_tot = im_tot + ims_pr
    
    counter = 0
    if im_tot == 0: # if no images
        download_frac = np.NaN
    else:
        # count the files that passed thresholds and got downloaded
        for file in os.listdir(downloadpath+'Box'+BoxID+'/reprojected/'):
            ######################################################################################
            if file.startswith('LC08') and file.endswith('.png') and 'B8' in file: # panchromatic band (B8)
            # adjust the if statement if you are using a different band
            #if file.startswith('LCXX') and file.endswith('.png') and 'BX' in file:
            ######################################################################################
                counter = counter + 1
            
        download_frac = int(counter/im_tot*100) # calculate fraction downloaded
    im_tots.append(im_tot); downloaded.append(counter); fractions.append(download_frac) # store values

# store in dataframe
downloaded_df = pd.DataFrame(list(zip(BoxIDs, im_tots, downloaded, fractions)), columns = ['BoxID', 'Total_ims', 'Downloaded', '%'])
downloaded_df 

,BoxID,Total_ims,Downloaded,%
0,008,1352,0,0


In [67]:
######################################################################################
DOWNLOADED_FILENAME = 'Images_downloaded_nonSE.csv' # change csv file name as desired
######################################################################################
downloaded_df.to_csv(basepath+DOWNLOADED_FILENAME, sep=',') # write to csv

# 7) Calculate weighted average glacier flow direction using velocity data

The following code processes ice velocity (vx, vy) rasters to determine each glacier of interest's weighted average flow direction. These files should be placed in the base directory (basepath). The rasters are subset using the terminus box shapefile or the Randolph Glacier Inventory outlines using a GDAL command (**gdalwarp**) with the following syntax:

    gdalwarp -cutline path_to_terminusbox.shp -crop_to_cutline path_to_input_velocity.TIF path_to_output_velocity_at_term###.TIF

In [69]:
######################################################################################
# Change to your velocity input file names
vx_name = 'greenland_vel_mosaic250_vx_v1.tif' # MEaSUREs product
vy_name = 'greenland_vel_mosaic250_vy_v1.tif' # MEaSUREs product

######################################################################################

for BoxID in BoxIDs:
    ######################################################################################
    # It's okay if you don't have the Randolph Glacier Inventory outlines, but if you do,
    # adjust the path to them here
    terminus_path = basepath+"Box"+BoxID+"/RGI_Box"+BoxID+".shp"  # path to RGI shapefiles
    
    ######################################################################################
    if os.path.exists(terminus_path) == False: # if the RGI shapefile does not exist
        terminus_path = basepath+"Box"+BoxID+"/Box"+BoxID+".shp"  # set the path to the terminux box shapefiles      
        # output paths for the cropped velocity data:
        vx_out = basepath+"Box"+BoxID+"/Box"+BoxID+'_'+vx_name
        vy_out = basepath+"Box"+BoxID+"/Box"+BoxID+'_'+vy_name
    else:
        vx_out = basepath+"Box"+BoxID+"/RGI_Box"+BoxID+'_'+vx_name
        vy_out = basepath+"Box"+BoxID+"/RGI_Box"+BoxID+'_'+vy_name
    
    # input paths:
    vx_in = basepath+vx_name
    vy_in = basepath+vy_name
    
    # subset x and y velocity files:
    v_subset1 = 'gdalwarp -cutline '+terminus_path+' -crop_to_cutline '+vx_in+" "+vx_out
    v_subset2 = 'gdalwarp -cutline '+terminus_path+' -crop_to_cutline '+vy_in+" "+vy_out
    subprocess.call(v_subset1, shell=True)
    subprocess.call(v_subset2, shell=True)
    
    print("Box"+BoxID+' done.')

Creating output file that is 59P x 50L.
Processing /Users/phoebekinzelman/research/greenland/updated_shps/greenland_vel_mosaic250_vx_v1.tif [1/1] : 0Using internal nodata values (e.g. -2e+09) for image /Users/phoebekinzelman/research/greenland/updated_shps/greenland_vel_mosaic250_vx_v1.tif.
Copying nodata values from source /Users/phoebekinzelman/research/greenland/updated_shps/greenland_vel_mosaic250_vx_v1.tif to destination /Users/phoebekinzelman/research/greenland/updated_shps/Box008/Box008_greenland_vel_mosaic250_vx_v1.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 59P x 50L.
Processing /Users/phoebekinzelman/research/greenland/updated_shps/greenland_vel_mosaic250_vy_v1.tif [1/1] : 0Using internal nodata values (e.g. -2e+09) for image /Users/phoebekinzelman/research/greenland/updated_shps/greenland_vel_mosaic250_vy_v1.tif.
Copying nodata values from source /Users/phoebekinzelman/research/greenland/updated_shps/greenland_vel_mosaic250_v

Next, these subset velocity rasters are opened using the **rasterio** package and read into arrays. They are filtered for anomalous values and the velocity magnitudes are converted into weights. Then the **numpy.average()** function is used to calculated the weighted average flow directions where the flow directions of the pixels where the highest velocities are found are weighted more. 

The resulting average flow direction will be representative of the glacier's main flow. These directions will be used to rotate the images of the glaciers so that their flow is due right.

__For slow-moving glaciers with uncertain velocities from feature tracking based velocity datasets, use manual determination of velocities. Here, we use the manual delineations of the Greenland peripheral glaciers in 2000 and 2015 to approximate the flow direction.__

In [70]:
##################################################################################################################
# ONLY APPLIES TO GREENLAND PERIPHERAL GLACIERS
badvelocities = ['301', '289', '283', '265', '241', '223', '285', '181', '097', '091', '067','083',
                 '221', '173', '113', '101', '089', '082', '100', '112', '118', '130', '160', '196', 
                 '208', '226', '256', '262', '280', '298', '322', '072', '074', '080', '082', '084',
                '102', '114', '134', '132', '144', '159', '188', '189', '198', '207', '212', '222',
                '224', '234', '242', '243', '249', '254', '258', '264', '267', '272', '273', '278', 
                '282', '284', '288', '297', '305', '306', '307', '315', '318', '321', '324', '327',
                '330', '331', '338', '341', '344', '354', '356', '357', '358', '359', '362', '363',
                 '364', '369', '370', '371', '372', '373', '374', '376', '377', '379', '380', '381', 
                 '382', '383', '384', '385', '386', '387', '388', '389', '390', '391', '392', '393',
                 '394', '395', '396', '397', '398', '399', '400', '401' ,'404', '405', '406', '407',
                 '408', '409', '410', '414', '415', '416', '417', '418', '419', '420', '421', '422',
                 '427', '430', '431', '434', '436', '438', '440']
print(badvelocities)
##################################################################################################################

['301', '289', '283', '265', '241', '223', '285', '181', '097', '091', '067', '083', '221', '173', '113', '101', '089', '082', '100', '112', '118', '130', '160', '196', '208', '226', '256', '262', '280', '298', '322', '072', '074', '080', '082', '084', '102', '114', '134', '132', '144', '159', '188', '189', '198', '207', '212', '222', '224', '234', '242', '243', '249', '254', '258', '264', '267', '272', '273', '278', '282', '284', '288', '297', '305', '306', '307', '315', '318', '321', '324', '327', '330', '331', '338', '341', '344', '354', '356', '357', '358', '359', '362', '363', '364', '369', '370', '371', '372', '373', '374', '376', '377', '379', '380', '381', '382', '383', '384', '385', '386', '387', '388', '389', '390', '391', '392', '393', '394', '395', '396', '397', '398', '399', '400', '401', '404', '405', '406', '407', '408', '409', '410', '414', '415', '416', '417', '418', '419', '420', '421', '422', '427', '430', '431', '434', '436', '438', '440']


In [74]:
boxes = []; avg_rot = []; max_mag = []; num_cells = []

for BoxID in BoxIDs:
    rot_angles = []; max_magnitudes = []
    
    # determine if RGI outline was used to subset velocities
    rgi_exists = 0
    for file in os.listdir(basepath+"Box"+BoxID):
        if file.startswith('RGI'):
            rgi_exists = 1
            
    if rgi_exists == 1: # if yes, open those files    
        vx = rio.open(basepath+"Box"+BoxID+"/RGI_Box"+BoxID+'_'+vx_name, "r") # RGI
        vy = rio.open(basepath+"Box"+BoxID+"/RGI_Box"+BoxID+'_'+vy_name, "r") # RGI
    else: # if not, they were subset using the boxes. Open those files
        vx = rio.open(basepath+"Box"+BoxID+"/Box"+BoxID+'_'+vx_name, "r") # box 
        vy = rio.open(basepath+"Box"+BoxID+"/Box"+BoxID+'_'+vy_name, "r") # box

    vx_array = vx.read(); vy_array = vy.read() # read as numpy array
    vx_masked = vx_array[vx_array != -2000000000.0] # remove no data values (-2000000000.0)
    vy_masked = vy_array[vy_array != -2000000000.0]

    direction = np.arctan2(vy_masked, vx_masked)*180/np.pi # calculate flow direction
    # transform so any negative angles are placed on 0 to 360 scale:
    if len(direction[direction < 0]) > 0:
        direction[direction < 0] = 360.0+direction[direction < 0]

    magnitude = np.sqrt((vx_masked*vx_masked) + (vy_masked*vy_masked)) # calculate speed (flow magnitude)
    
    ncells = len(direction) # number of pixels
    if ncells > 0:
        # Determine if there are a large number of direction pixels with values > 200.0
        # If so, it's probably pointing East
        dir_range = direction.max() - direction.min()
        if dir_range > 200.0 and len(direction[direction > 200]): # if large range and values above 200
            direction[direction > 180] = direction[direction > 180] - 360.0 # transform those values on a negative scale
            # calculate weights (0 - 1) from magnitudes
            mag_range = magnitude.max() - magnitude.min()
            stretch = 1/mag_range; weights = stretch*(magnitude - magnitude.min()) # weights for averaging
            avg_dir = np.average(direction, weights=weights) # calculate average flow direction
            if avg_dir < 0: # if negative:
                avg_dir = avg_dir + 360.0 # transform back to 0 to 360 scale
        else:
            mag_range = magnitude.max() - magnitude.min(); stretch = 1/mag_range
            weights = stretch*(magnitude - magnitude.min())
            avg_dir = np.average(direction, weights=weights)
                
        ###############################################################################################################
        # Adjust to convert flow magnitude to meters per day depending on units in your 
        # velocity dataset
        yr_day_conv = 0.00273973 # conversion to m/d from m/a
        max_magnitude = magnitude.max()*yr_day_conv 
        ###############################################################################################################
        
    else: # no velocity pixels remaining once cropped
        avg_dir = np.NaN ; max_magnitude = np.NaN # no velocities to calculate this with
    
    ##################################################################################################################
    # ONLY APPLIES TO GREENLAND PERIPHERAL GLACIERS, COMMENT OUT FOR OTHER APPLICATIONS
    path2000_2015 = '/phoebekinzelman/research/greenland/2000_2015/'
    if BoxID in badvelocities:
        # grab the 2000 and 2015 delineation centroids:
        shp2000 = fiona.open(path2000_2015+'GreenlandPeriph_term2000_'+BoxID+'.shp'); feat2000= shp2000.next()
        lineshp2000 = LineString(feat2000['geometry']['coordinates'])
        cent2000 = np.array(lineshp2000.centroid)

        shp2015 = fiona.open(path2000_2015+'GreenlandPeriph_term2015_'+BoxID+'.shp'); feat2015= shp2015.next()
        lineshp2015 = LineString(feat2015['geometry']['coordinates'])
        cent2015 = np.array(lineshp2015.centroid)

        # grab displacements and use to calculate flow direction in degrees
        y = cent2000[1] - cent2015[1]
        x = cent2000[0] - cent2015[0]
        avg_dir = np.arctan2(y,x)*180/np.pi
        if avg_dir < 0:
            avg_dir = 360.0+avg_dir
         
        # if max_magnitude cannot be calculated from the velocity raster (pixels == 0)
        if ncells == 0:
            # use displacements and time to approximate speed in m/d
            yrs = 15.0
            max_magnitude = np.sqrt((y*y)+(x*x))/yrs*yr_day_conv
                
        ncells = np.NaN
    ##################################################################################################################
    
    # Append values to lists:
    avg_rot.append(avg_dir); max_mag.append(max_magnitude); boxes.append(BoxID); num_cells.append(ncells)  

# store the flow direction (rotation angle), maximum magnitude
velocities_df = pd.DataFrame(list(zip(boxes,avg_rot, max_mag, num_cells)), columns=['BoxID','Flow_dir', 'Max_speed', 'Pixels'])
velocities_df = velocities_df.sort_values(by='BoxID')
velocities_df # display

,BoxID,Flow_dir,Max_speed,Pixels
0,008,316.004959,4.696953,429


In [75]:
##################################################################################################################
VEL_FILENAME = 'Glacier_vel_Tess.csv' # change velocity file name
##################################################################################################################
velocities_df.to_csv(path_or_buf = basepath+VEL_FILENAME, sep=',') # write to csv

# 8) Rotate all images by flow direction

Read in the glacier velocity file as velocities_df if not already loaded:

In [76]:
velocities_df = pd.read_csv(basepath+VEL_FILENAME, sep=',', dtype=str, usecols=[1,2,3,4])
velocities_df = velocities_df.set_index('BoxID')

In [77]:
# make directory for rotated images in BoxID folders if it doesn't already exist
for index, row in velocities_df.iterrows():
    BoxID = index
    if os.path.exists(downloadpath+"Box"+BoxID+'/rotated_c1/'):
        print("Already exists.")
    else:
        os.mkdir(downloadpath+"Box"+BoxID+'/rotated_c1/')
        print("Folder made for Box"+BoxID)

Folder made for Box008


In [78]:
# move rasterized terminus box into reprojected folder, since it will also need to be rotated:
for index, row in velocities_df.iterrows():
    BoxID = index
    boxfile = 'Box'+BoxID+'_raster_cut.TIF'
    boxrasterpath = basepath+'Box'+BoxID+'/'+boxfile
    newpath = downloadpath+'Box'+BoxID+'/reprojected/'+boxfile
    shutil.copyfile(boxrasterpath, newpath)

In [94]:
# convert all images to png for rotation:
for index, row in velocities_df.iterrows():
    BoxID = index
    command = 'cd '+downloadpath+'Box'+BoxID+'/reprojected/; '+'mogrify -format png *raster_cut.TIF'
    print(command)
    subprocess.call(command, shell=True)

cd /Users/phoebekinzelman/research/greenland/LS8aws/Box008/reprojected/; mogrify -format png *raster_cut.TIF


/bin/sh: mogrify: command not found


In [92]:
# rotate
for index, row in velocities_df.iterrows():
    BoxID = index
    print(BoxID) # keep track of progress
    for file in os.listdir(downloadpath+"Box"+BoxID+'/reprojected/'):
        if file.endswith('.png'):
            img  = Image.open(downloadpath+"Box"+BoxID+'/reprojected/'+file)
            rotated = img.rotate(-float(row['Flow_dir']))
            rotated.save(downloadpath+"Box"+BoxID+'/rotated_c1/R_'+file)

008


# 9) Crop all images to the same size

All input images will need to be the same size for the automated terminus detection analysis. The function resize_pngs resizes all png files within a folder to the minimum image dimensions found. The function crops around the edges, centering the image.

In [93]:
for BoxID in BoxIDs:
    resizepath = downloadpath+"Box"+BoxID+'/rotated_c1/' # path to rotated images
    resize_pngs(resizepath) # crop

ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
##################################################################################################################
# FOR XSMURF ANALYSIS ONlY
# convert all final files to pgm
for index, row in velocities_df.iterrows():
    BoxID = index
    command = 'cd '+downloadpath+'Box'+BoxID+'/rotated_c1/; '+'mogrify -depth 16 -format pgm *.png'
    subprocess.call(command, shell=True)
##################################################################################################################

In [ ]:
# # rename the rasterized terminus box files if necessary
# for BoxID in BoxIDs:
#     files = os.listdir(downloadpath+'Box'+BoxID+'/rotated_c1/')
#     for file in files:
#         if file.startswith('R_Box'+BoxID+'_cut'):
#             rpath = downloadpath+'Box'+BoxID+'/rotated_c1/'
#             os.rename(rpath+file, rpath+'R_Box'+BoxID+'_raster_cut'+file[-4:])